In [1]:
from model import SimpleView
from dataset import MpalaTreeLiDAR
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import optim, nn
import d_util

training_data = MpalaTreeLiDAR(
    data_path='../data/MpalaForestGEO_LasClippedtoTreePolygons',
    ids=range(1, 30),
    transform=transforms.Compose([
        d_util.ToPointCloud(),
        d_util.ProjectPointCloud(),
        d_util.ExpandChannels(channels=3),
    ]),
)

trainloader = DataLoader(training_data, batch_size=3, shuffle=True)


simpleview = SimpleView(num_views=6, num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(simpleview.parameters(), lr=0.001, momentum=0.9)
epochs = 100

for _ in range(epochs):
    for x, y in trainloader:
        optimizer.zero_grad()

        outputs = simpleview(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        print(x.shape)
        print(simpleview(x).shape)
        break
    # when you ask for a 1 sentence pitch, it's you saying you don't want to understand the problem, but digest it for entertainment like its a tiktok

# epochs = 100
# for epoch in range(epochs):  # loop over the dataset multiple times
#     running_loss = 0.0
#     for i, data in enumerate(trainloader):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data
#         # zero the parameter gradients
#         optimizer.zero_grad()
#         # forward + backward + optimize
#         outputs = simpleview(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         # print statistics
#         running_loss += loss.item()

torch.Size([3, 6, 3, 244, 244])
torch.Size([3, 4])
torch.Size([3, 6, 3, 244, 244])
